In [1]:
import numpy as np
import pandas as pd

# Create Data

In [162]:
data = np.array([[12, 411, 3, 4.4, 5, 10,27 ],
            [14, 430, 4, 6, 5, 10,20 ],
            [12, 411, 3.1, 4, 9, 5,10 ],
            [14, 413, 4.1, 4.4, 7, 5,11 ]])
total_columns = [ 'T','score', 'var1', 'var2','var3', 'var4','var5']
df = pd.DataFrame(data=data, columns=total_columns)
df

,T,score,var1,var2,var3,var4,var5
0,12.0,411.0,3.0,4.4,5.0,10.0,27.0
1,14.0,430.0,4.0,6.0,5.0,10.0,20.0
2,12.0,411.0,3.1,4.0,9.0,5.0,10.0
3,14.0,413.0,4.1,4.4,7.0,5.0,11.0


In [119]:
data = np.array([[ 0.1, 0.2,  0.3 ]])
columns = [ 'var1', 'var3','var5']
df_coefs12 = pd.DataFrame(data=data, columns=columns)
df_coefs12

,var1,var3,var5
0,0.1,0.2,0.3


In [201]:
data = np.array([[ 0.4, 0.8,  1 ]])
columns = [ 'var2', 'var4','var5']
df_coefs14 = pd.DataFrame(data=data, columns=columns)
df_coefs14

,var2,var4,var5
0,0.4,0.8,1.0


In [185]:
def dot_product(x,columns,coefs='all_coefs'):
    """Computes X*b for each element in a dataframe
    """
    X = x[columns].values.reshape(-1,1)  # one row of variables is taken
    beta = x[coefs].reshape(-1,1) # complete coefficients are assigned to beta
    np.dot(beta.T,X) # dot product is computed
    return  np.dot(beta.T,X)[0][0]

def get_np_coefs(df_coefs,df):
    """Completes variable names in a pandas dataframe. Returns a dataframe with coefficients

    Arguments:
    df_coefs -- a pandas dataframe containing only one row with model coefficients. (df_coefs.columns is a subset of df.columns)
    df -- a pandas dataframe from which we take the df.columns values as template to complete coefficients

    Example: 
    data = np.array([[12, 411, 3, 4.4, 5, 10,27 ],
            [14, 430, 4, 6, 5, 10,20 ]])
    columns = [ 'x','y', 'var1', 'var2','var3', 'var4','var5']
    df = pd.DataFrame(data=data, columns=columns)

    data = np.array([[ 0.1, 0.2,  0.3 ]])
    columns = [ 'var1', 'var3','var5']
    df_coefs = pd.DataFrame(data=data, columns=columns)

    completed_coefs = get_np_coefs(df_coefs14,df)

    Which returns:

        x	y	var1	var2	var3	var4	var5	all_coefs
    0	0.0	0.0	0.0	    0.4	    0.0	    0.8	    1.0	    [[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]

    """
    coef_list = [df_coefs.loc[0,var_name] if (var_name in df_coefs.columns) else 0 for var_name in df.columns]
    coef_list = np.array(coef_list).reshape(1,-1)
    df_coef =pd.DataFrame(data=coef_list,columns=df.columns)
    df_coef['all_coefs'] = [coef_list]
    return df_coef

In [186]:
# Coefficients for T=12 and T=14
beta12 = get_np_coefs(df_coefs12,df)
beta12['joinVar'] =12
beta14 = get_np_coefs(df_coefs14,df)
beta14['joinVar'] =14
df_concat = pd.concat([beta12,beta14],ignore_index=True)
df_concat

,T,score,var1,var2,var3,var4,var5,all_coefs,joinVar
0,0.0,0.0,0.1,0.0,0.2,0.0,0.3,"[[0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.3]]",12
1,0.0,0.0,0.0,0.4,0.0,0.8,1.0,"[[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]",14


In [187]:
# Joining coeff dataframe with complete df
df_complete = pd.merge(df,df_concat[['all_coefs','joinVar']],left_on='T',right_on='joinVar')
df_complete

,T,score,var1,var2,var3,var4,var5,all_coefs,joinVar
0,12.0,411.0,3.0,4.4,5.0,10.0,27.0,"[[0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.3]]",12
1,12.0,411.0,3.1,4.0,9.0,5.0,10.0,"[[0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.3]]",12
2,14.0,430.0,4.0,6.0,5.0,10.0,20.0,"[[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]",14
3,14.0,413.0,4.1,4.4,7.0,5.0,11.0,"[[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]",14


In [200]:
# compute dot product
df_complete['res1'] = df_complete.apply(lambda x : dot_product(x,total_columns),axis=1)
df_complete

,T,score,var1,var2,var3,var4,var5,all_coefs,joinVar,res1
0,12.0,411.0,3.0,4.4,5.0,10.0,27.0,"[[0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.3]]",12,9.40
1,12.0,411.0,3.1,4.0,9.0,5.0,10.0,"[[0.0, 0.0, 0.1, 0.0, 0.2, 0.0, 0.3]]",12,5.11
2,14.0,430.0,4.0,6.0,5.0,10.0,20.0,"[[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]",14,30.40
3,14.0,413.0,4.1,4.4,7.0,5.0,11.0,"[[0.0, 0.0, 0.0, 0.4, 0.0, 0.8, 1.0]]",14,16.76
